In [1]:
import os
import cv2
import pandas as pd
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

## EXPERIMENT 1: MATRIU DE PÍXELS

In [7]:
# Script de python per convertir les imatges a informació per pixel i etiqueta

folder_train = "dat/a2/data/train"
folder_test = "dat/a2/data/test"

# Initialize train and test dataframes
df_train = None
df_test = None

data = []

# Tamaño deseado para las imágenes (imagen mínima)
desired_size = (200, 200)

def data_to_pixels(base_folder):
    global df_train, df_test
    
    for clase in os.listdir(base_folder):
        clase_folder = os.path.join(base_folder, clase)
        if os.path.isdir(clase_folder):
            for imagen_filename in os.listdir(clase_folder):
                if imagen_filename.endswith(".jpg") or imagen_filename.endswith(".png"):
                    imagen_path = os.path.join(clase_folder, imagen_filename)
                    # Carga la imagen en escala de grises
                    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
                    # Redimensiona la imagen al tamaño deseado
                    imagen_resized = cv2.resize(imagen, desired_size)
                    # Aplana la imagen en un solo vector de píxeles
                    pixels = imagen_resized.flatten()
                    # Agrega la fila de datos al conjunto de datos
                    data.append([clase] + list(pixels))

    if base_folder == "dat/a2/data/train":
        column_names = ['label'] + [f'pixel{i}' for i in range(1, len(data[0]))]
        df_train = pd.DataFrame(data, columns=column_names)
    else:
        column_names = ['label'] + [f'pixel{i}' for i in range(1, len(data[0]))]
        df_test = pd.DataFrame(data, columns=column_names)

data_to_pixels(folder_train)
data_to_pixels(folder_test)

# Now, df_train and df_test are initialized with the data


In [8]:
# Training and testing data info
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Columns: 40001 entries, label to pixel40000
dtypes: object(1), uint8(40000)
memory usage: 8.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Columns: 40001 entries, label to pixel40000
dtypes: object(1), uint8(40000)
memory usage: 17.7+ MB


In [9]:
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel39991,pixel39992,pixel39993,pixel39994,pixel39995,pixel39996,pixel39997,pixel39998,pixel39999,pixel40000
0,bedroom,156,128,96,76,74,81,87,86,86,...,43,46,48,41,48,42,37,34,32,35
1,bedroom,204,182,180,178,178,177,176,176,176,...,32,30,30,28,25,26,26,25,25,24
2,bedroom,217,216,216,217,215,217,218,217,219,...,193,196,197,197,197,197,194,188,193,195
3,bedroom,50,54,58,58,57,56,56,61,62,...,19,18,15,12,11,6,2,1,0,0
4,bedroom,218,223,225,228,231,232,234,235,234,...,255,255,255,255,254,255,255,254,255,255


In [10]:
df_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel39991,pixel39992,pixel39993,pixel39994,pixel39995,pixel39996,pixel39997,pixel39998,pixel39999,pixel40000
0,bedroom,156,128,96,76,74,81,87,86,86,...,43,46,48,41,48,42,37,34,32,35
1,bedroom,204,182,180,178,178,177,176,176,176,...,32,30,30,28,25,26,26,25,25,24
2,bedroom,217,216,216,217,215,217,218,217,219,...,193,196,197,197,197,197,194,188,193,195
3,bedroom,50,54,58,58,57,56,56,61,62,...,19,18,15,12,11,6,2,1,0,0
4,bedroom,218,223,225,228,231,232,234,235,234,...,255,255,255,255,254,255,255,254,255,255


In [11]:
# Split data into features (X) and labels (y)
X_train = df_train.iloc[:, 1:].values  # Features (pixels)
y_train = df_train.iloc[:, 0].values  # Labels

X_test = df_test.iloc[:, 1:].values  # Test features
y_test = df_test.iloc[:, 0].values  # Test labels

print("Splitted labels (X) and pixels(y)")

Splitted labels (X) and pixels(y)


In [15]:
# Create an SVM classifier with a linear kernel
clf = SVC(kernel='linear')

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Test Accuracy:", accuracy)
print("Classification Report:\n", report)

Test Accuracy: 0.4967741935483871
Classification Report:
                             precision    recall  f1-score   support

                     Coast       0.28      1.00      0.43        15
                    Forest       0.54      1.00      0.70        15
                   Highway       0.37      1.00      0.54        15
                Insidecity       0.62      1.00      0.77        15
                  Mountain       0.56      1.00      0.71        15
                    Office       0.50      1.00      0.67        15
               OpenCountry       0.26      1.00      0.41        15
                    Street       0.60      1.00      0.75        15
                    Suburb       0.60      1.00      0.75        15
              TallBuilding       0.52      1.00      0.68        15
                   bedroom       0.80      0.53      0.64        30
                     coast       0.00      0.00      0.00        15
                    forest       0.00      0.00      0.00

d:\python\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## EXPERIMENT 2: HOG

In [ ]:
# Define constants

folder_train = "dat/a2/data/train"
folder_test = "dat/a2/data/test"

image_size = (200, 200)
orientations = 8
pixels_per_cell = (12, 12)
cells_per_block = (1, 1)

# Function to extract HOG features from an image
def extract_hog_features(image):
    fd, _ = hog(
        image,
        orientations=orientations,
        pixels_per_cell=pixels_per_cell,
        cells_per_block=cells_per_block,
        visualize=True,
    )
    return fd

# Function to load and preprocess images
def load_and_preprocess_images(folder_path):
    X = []
    y = []

    for class_label, class_name in enumerate(os.listdir(folder_path)):
        class_folder = os.path.join(folder_path, class_name)
        if os.path.isdir(class_folder):
            for image_filename in os.listdir(class_folder):
                if image_filename.endswith(".jpg") or image_filename.endswith(".png"):
                    image_path = os.path.join(class_folder, image_filename)

                    # Load and resize the image
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    image = cv2.resize(image, image_size)

                    # Extract HOG features
                    hog_features = extract_hog_features(image)

                    X.append(hog_features)
                    y.append(class_label)
            print("***************************CARPETA TERMINADA***********************************")

    return X, y

# Load and preprocess training and testing data
X_train, y_train = load_and_preprocess_images(folder_train)
print("**TRAIN TERMINADO**")
X_test, y_test = load_and_preprocess_images(folder_test)
print("**TEST TERMINADO**")

### SVM 'linear' with HOG

In [ ]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a linear SVM model
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

### Búsqueda de hiperparámetros

In [ ]:
# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10, 100],  # Different values of the regularization parameter C
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Different kernel functions to try
    'gamma': []
}

# Create an SVM classifier
svm = SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=5, n_jobs=-1)  # cv is the number of cross-validation folds
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

### SVM con mejores hiperparámetros

In [ ]:
# Train the SVM model with the best hyperparameters on the entire training set
best_svm = SVC(**best_params)
best_svm.fit(X_train, y_train)

# Make predictions on the test data
y_test_pred = best_svm.predict(X_test)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)